In [1]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split,StratifiedKFold # Model evaluation
from sklearn.preprocessing import LabelEncoder, RobustScaler, OneHotEncoder, StandardScaler # Preprocessing
from sklearn.linear_model import Lasso, Ridge, ElasticNet,  LassoLarsIC, RANSACRegressor, SGDRegressor, HuberRegressor, BayesianRidge # Linear models
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor  # Ensemble methods
from xgboost import XGBRegressor, plot_importance # XGBoost
from sklearn.svm import SVR, SVC, LinearSVC  # Support Vector Regression
from sklearn.tree import DecisionTreeRegressor # Decision Tree Regression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline # Streaming pipelines
from sklearn.decomposition import KernelPCA, PCA # Dimensionality reduction
from sklearn.feature_selection import SelectFromModel # Dimensionality reduction
from sklearn.model_selection import learning_curve, validation_curve, GridSearchCV # Model evaluation
from sklearn.base import clone, BaseEstimator, TransformerMixin, RegressorMixin # Clone estimator
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import explained_variance_score, f1_score, roc_auc_score, median_absolute_error, r2_score, mean_squared_error #To evaluate our model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, fbeta_score #To evaluate our model
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split 
import optuna

In [2]:
import pandas as pd
import numpy as np
import re
import numpy as np

In [3]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
testId = test.Property_Id
train.Price_Category = train.Price_Category.replace("Cheap",0).replace("Affordable",1).replace("Semi-Premium",2).replace("Premium",3)
y = train.Price_Category
train = train.drop(["Property_Id", "Price_Category"],axis=1)
test = test.drop(["Property_Id"],axis=1)
features = pd.concat([train,test])
train.head(1)

,Title,Label,Type,Location,Baths,Area,Purpose,Bedroom(s),Description,Main Features,Rooms,Business and Communication,Healthcare Recreational,Nearby Locations and Other Facilities,Other Facilities,Popular,Source,Time Stamp
0,Brand New 1 Kanal House For Sale In Bahria Tow...,"Bahria Town - Gulbahar Block, Bahria Town - Se...",House,"Bahria Town, Lahore, Punjab",6,1 Kanal,For Sale,5,Brand New 1 Kanal House For Sale In Bahria Tow...,"Built in year: 2018 , Parking Spaces: 4 , Dou...","Bedrooms: 5 , Bathrooms: 7 , Servant Quarters...","Broadband Internet Access , Satellite or Cabl...","Lawn or Garden , Swimming Pool","Nearby Schools , Nearby Hospitals , Nearby Sh...","Maintenance Staff , Security Staff , Faciliti...",super hot,Lahore-1-10,3/31/2019 11:01


In [4]:
def findWcolon(a,x, length):
    val = re.findall("%s: \d+" % a,x)
    val = val[0][-length:] if len(val)>0 else (np.nan if len(x)==1 else 0)
    return val
def find(a,x):
    return 1 if a in x else (np.nan if len(x)==1 else 0)

def rooms(x):
    living = find("Study Room",x) + find("Drawing Room",x) + find("Dining Room",x) + find("Lounge or Sitting Room", x)
    athletic = find("Gym",x) + find("Steam Room",x) + find("Laundry Room",x) + find("Powder Room", x)
    #findWcolon("Bathrooms",x,1),
    return [findWcolon("Servant Quarters",x,1), findWcolon("Kitchens",x,1), findWcolon("Store Rooms",x,1), living, athletic, find("Other Rooms", x), find("Steam Room",x), find("Laundry Room",x), find("Powder Room", x), find("Study Room",x), find("Drawing Room",x), find("Dining Room",x), find("Lounge or Sitting Room", x)]

def main_features(x):
    return [findWcolon("Built in year",x,4), findWcolon("Parking Spaces",x,1), findWcolon("Floors",x,1), find("Double Glazed Windows", x), find("Flooring", x), find("Electricity Backup", x), find("Waste Disposal", x),find("Central Air Conditioning", x), find("Flooring", x)]

def business(x):
    return [find("Broadband Internet Access", x), find("Satellite or Cable TV Ready", x), find("Intercom", x), find("Other Business and Communication Facilities", x)]

def health(x):
    return [find("Lawn or Garden", x), find("Sauna", x) + find("Jacuzzi", x), find("Swimming Pool", x), find("Other Healthcare and Recreation Facilities", x)]

def facilities(x):
    cols = ['Maintenance Staff','Security Staff','Facilities for Disabled','Other Facilities']
    return [find(y,x) for y in cols]

def nearby(x):
    val = len([y for y in x.split(" , ") if "Nearby" in y]) if x != " " else np.nan
    return val


def process(train):
    train["nearbyVal"] = [1 if len(x) > 1 else 0 for x in train["Nearby Locations and Other Facilities"]]
    train["bussinessVal"] = [1 if len(x) > 1 else 0 for x in train["Business and Communication"]]
    train["staffVal"] = [1 if len(x) > 1 else 0 for x in train["Other Facilities"]]
    train["healthVal"] = [1 if len(x) > 1 else 0 for x in train["Healthcare Recreational"]]
    train[["Date","Parking","Floors","Windows", "Flooring", "ElectricBackup", "WasteDisposal", "CentralAir", "CentralHeating"]] = list(train["Main Features"].apply(main_features))
    train[["Servants","Kitchens","Store Rooms", "living","healthRooms","OtherRooms", "Steam Room","Laundry Room","Powder Room","Study Room","Drawing Room","Dining Room","Lounge"]] = list(train.Rooms.apply(rooms))
    train[["internet","tv","intercom","otherBusiness"]] = list(train["Business and Communication"].apply(business))
    train[["town","city","province"]] = list(train.Location.apply(lambda x: x.split(",")))
    train[["lawn","hotTub","pool","otherHealth"]] = list(train["Healthcare Recreational"].apply(health))
    train.Area = ["".join(x.split(",")) for x in train.Area]
    train.Area = train.Area.apply(lambda x: float(x.split(" ")[0]) if x.split(" ")[1] == "Marla" else float(x.split(" ")[0]) *20)
    train[['MaintenanceStaff','SecurityStaff','FacilitiesDisabled','ExtraFacilities']] = list(train["Other Facilities"].apply(facilities))
    train["nearby2"] = train["Nearby Locations and Other Facilities"].apply(nearby)
    train.Popular = train.Popular.replace("hot",1).replace("super hot",2)
    train = pd.concat([train,pd.get_dummies(train.Location)],axis=1)
    train[["Source", "SourceNum"]] = [["-".join(x.split("-")[:-1]), x.split("-")[-1] ]for x in train.Source]
    train = pd.concat([train,pd.get_dummies(train.Source)],axis=1)
    train = train.replace("-",0)
    train = train.drop(["Title","Label","Date", "city", "province", "town","Type","Purpose", "Healthcare Recreational", "Other Facilities", "Rooms","Nearby Locations and Other Facilities","Main Features", "Source", "Time Stamp", "Description", "Location", "Business and Communication"],axis=1)
    train = train.fillna(0)
    train = train.astype('int')
    return train

In [5]:
features = process(features)
train = features[:len(y)]
test = features[len(y):]
train = train.values
test = test.values


In [36]:
#Set to False if you want to skip it
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.33, random_state=42)
OPTUNA_OPTIMIZATION = True
N_SPLITS = 10 #Number of folds for validation
N_TRIALS = 100 #Number of trials to find best hyperparameters

In [37]:
def objective(trial, cv=StratifiedKFold(N_SPLITS, shuffle = True, random_state = 29)):
    
    
    param_lgb = {
        "random_state": trial.suggest_int("random_state", 1, 100),
        "objective": "multiclass",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "max_depth": trial.suggest_int("max_depth", -1, 10),
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.4, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    
    model = LGBMClassifier(**param_lgb)
    
    val_f1 = []
    f1s = []
    #skf = StratifiedKFold(N_SPLITS, shuffle = True, random_state = 29)
    #for kfold, (train_idx, val_idx) in enumerate(skf.split(train, y)):
    for kfold, (train_idx, val_idx) in enumerate(cv.split(train, y)):
        
        model.fit(train[train_idx], y[train_idx])
        print('Fitted {}'.format(type(model).__name__))
        val_true = y[val_idx].values
        
        preds = model.predict(train[val_idx]).round(0)
        
        f1 = f1_score(val_true, preds, average='macro')
        
        print('Fold: {}\t F1: {}\n'.format(kfold, f1))
        f1s.append(f1)
    
    print('Average F1: {}'.format(np.average(f1)))
    return np.average(f1)

In [38]:
if OPTUNA_OPTIMIZATION:
    study = optuna.create_study(study_name = 'lgbm_parameter_opt', direction="maximize")
    study.optimize(objective, n_trials=N_TRIALS) 
    
    trial = study.best_trial
    
    print("  Value: {}".format(trial.value))
    
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
else:
    trial = {"reg_alpha": 0.362136938773081,
             "reg_lambda": 2.930297242488071,
             "max_depth": 10,
             "n_estimators": 306,
             "num_leaves": 71,
             "colsample_bytree": 0.7121396258381646,
             "subsample": 0.793959734582999,
             "subsample_freq": 2,
             "min_child_samples": 18}

[I 2021-03-22 22:02:24,254] A new study created in memory with name: lgbm_parameter_opt


Fitted LGBMClassifier
Fold: 0	 F1: 0.6213454657193315

Fitted LGBMClassifier
Fold: 1	 F1: 0.7712841397772905

Fitted LGBMClassifier
Fold: 2	 F1: 0.5919841822522881

Fitted LGBMClassifier
Fold: 3	 F1: 0.7497626538610145

Fitted LGBMClassifier
Fold: 4	 F1: 0.5898209332680698

Fitted LGBMClassifier
Fold: 5	 F1: 0.6064306064306064

Fitted LGBMClassifier
Fold: 6	 F1: 0.6325621776211634

Fitted LGBMClassifier
Fold: 7	 F1: 0.7182594644506002

Fitted LGBMClassifier
Fold: 8	 F1: 0.6608267561168114



[I 2021-03-22 22:03:01,317] Trial 0 finished with value: 0.742787296802788 and parameters: {'random_state': 42, 'reg_alpha': 6.399209615276174e-08, 'reg_lambda': 3.5747809318337627, 'max_depth': 2, 'n_estimators': 279, 'num_leaves': 200, 'colsample_bytree': 0.9387664767067961, 'subsample': 0.7612317922450208, 'subsample_freq': 4, 'min_child_samples': 61}. Best is trial 0 with value: 0.742787296802788.


Fitted LGBMClassifier
Fold: 9	 F1: 0.742787296802788

Average F1: 0.742787296802788
Fitted LGBMClassifier
Fold: 0	 F1: 0.6130523138832997

Fitted LGBMClassifier
Fold: 1	 F1: 0.7413347192832659

Fitted LGBMClassifier
Fold: 2	 F1: 0.5818662354265604

Fitted LGBMClassifier
Fold: 3	 F1: 0.85271546635183

Fitted LGBMClassifier
Fold: 4	 F1: 0.59700399738946

Fitted LGBMClassifier
Fold: 5	 F1: 0.5928415260992745

Fitted LGBMClassifier
Fold: 6	 F1: 0.654556118973738

Fitted LGBMClassifier
Fold: 7	 F1: 0.7263433658341691

Fitted LGBMClassifier
Fold: 8	 F1: 0.5722782019329447



[I 2021-03-22 22:03:48,871] Trial 1 finished with value: 0.6886320406051348 and parameters: {'random_state': 41, 'reg_alpha': 0.012269866884074537, 'reg_lambda': 0.04490744929017145, 'max_depth': 1, 'n_estimators': 288, 'num_leaves': 151, 'colsample_bytree': 0.45754526356152103, 'subsample': 0.6943197345526191, 'subsample_freq': 1, 'min_child_samples': 55}. Best is trial 0 with value: 0.742787296802788.


...

Fitted LGBMClassifier
Fold: 9	 F1: 0.7635911307019302

Average F1: 0.7635911307019302
Fitted LGBMClassifier
Fold: 0	 F1: 0.554212931548537

Fitted LGBMClassifier
Fold: 1	 F1: 0.7010877526519096

Fitted LGBMClassifier
Fold: 2	 F1: 0.5997150997150997

Fitted LGBMClassifier
Fold: 3	 F1: 0.7468159597147688

Fitted LGBMClassifier
Fold: 4	 F1: 0.6038168182963567

Fitted LGBMClassifier
Fold: 5	 F1: 0.5693146417445483

Fitted LGBMClassifier
Fold: 6	 F1: 0.6500687666463068

Fitted LGBMClassifier
Fold: 7	 F1: 0.7678485036471193



[I 2021-03-22 22:36:44,940] Trial 98 finished with value: 0.7661939755412761 and parameters: {'random_state': 66, 'reg_alpha': 0.000665066847088508, 'reg_lambda': 0.029350095423818345, 'max_depth': 1, 'n_estimators': 114, 'num_leaves': 80, 'colsample_bytree': 0.7507356129759164, 'subsample': 0.6789946840416237, 'subsample_freq': 7, 'min_child_samples': 7}. Best is trial 96 with value: 0.8794582532696957.


Fitted LGBMClassifier
Fold: 8	 F1: 0.7177750085520207

Fitted LGBMClassifier
Fold: 9	 F1: 0.7661939755412761

Average F1: 0.7661939755412761
Fitted LGBMClassifier
Fold: 0	 F1: 0.634520711342442

Fitted LGBMClassifier
Fold: 1	 F1: 0.741324242794831

Fitted LGBMClassifier
Fold: 2	 F1: 0.6022846445947971

Fitted LGBMClassifier
Fold: 3	 F1: 0.7773598820058998

Fitted LGBMClassifier
Fold: 4	 F1: 0.624281130445514

Fitted LGBMClassifier
Fold: 5	 F1: 0.581587947379798

Fitted LGBMClassifier
Fold: 6	 F1: 0.6426245691820087

Fitted LGBMClassifier
Fold: 7	 F1: 0.7173245614035088

Fitted LGBMClassifier
Fold: 8	 F1: 0.6039506324702216



[I 2021-03-22 22:36:47,378] Trial 99 finished with value: 0.6888817065287653 and parameters: {'random_state': 46, 'reg_alpha': 0.0019130194488910608, 'reg_lambda': 6.515612874540475e-07, 'max_depth': 2, 'n_estimators': 150, 'num_leaves': 108, 'colsample_bytree': 0.7364886284759814, 'subsample': 0.6482510135617201, 'subsample_freq': 6, 'min_child_samples': 23}. Best is trial 96 with value: 0.8794582532696957.


Fitted LGBMClassifier
Fold: 9	 F1: 0.6888817065287653

Average F1: 0.6888817065287653
  Value: 0.8794582532696957
  Params: 
    random_state: 61
    reg_alpha: 0.007077185361624641
    reg_lambda: 9.387100827254451e-07
    max_depth: 2
    n_estimators: 135
    num_leaves: 90
    colsample_bytree: 0.7487206162142912
    subsample: 0.7253774939270248
    subsample_freq: 7
    min_child_samples: 11


In [11]:
if OPTUNA_OPTIMIZATION:
    final_model = LGBMRegressor(**trial.params)
else:
    final_model = LGBMRegressor(**trial)

In [13]:
trial.params

{'random_state': 3,
 'reg_alpha': 0.6277984474689421,
 'reg_lambda': 1.087969024197104e-06,
 'max_depth': -1,
 'n_estimators': 463,
 'num_leaves': 204,
 'colsample_bytree': 0.790826582771263,
 'subsample': 0.48077060872853045,
 'subsample_freq': 6,
 'min_child_samples': 30}

In [18]:
test_preds = []

skf = StratifiedKFold(N_SPLITS, shuffle = True, random_state = 29)
for kfold, (train_idx, val_idx) in enumerate(skf.split(train, y)):

    final_model.fit(train[train_idx], y[train_idx])
    
    val_true = y[val_idx].values

    preds = final_model.predict(train[val_idx]).round(0)

    f1 = f1_score(val_true, preds, average='macro')

    print('Fold: {}\t F1: {}\n'.format(kfold, f1))

    test_preds.append(final_model.predict(test))
        


Fold: 0	 F1: 0.5845053320663077

Fold: 1	 F1: 0.709194313454018

Fold: 2	 F1: 0.47116612442815653

Fold: 3	 F1: 0.8562973687973687

Fold: 4	 F1: 0.5937438752263356

Fold: 5	 F1: 0.6089247702611494

Fold: 6	 F1: 0.6309480676328503

Fold: 7	 F1: 0.7220158422988612

Fold: 8	 F1: 0.5852205605127561

Fold: 9	 F1: 0.7847164065473925



In [25]:
preds = np.array(test_preds).mean(axis=0).round(0)

In [8]:
def run_model(name, model,train,y,test):
    X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.33, random_state=42)
    model2 = model
    model2.fit(X_train,y_train)
    predictions = model.predict(X_test).round(0)
    print("%s F1 %s" % (name,f1_score(y_test, predictions, average='macro')))
    model.fit(train,y)
    preds = model.predict(test).round(0)
    return preds

In [9]:
models = [
    ("KNN",KNeighborsClassifier(n_neighbors=5)),
    ("LGBM",LGBMRegressor()),
    ("XGB",XGBRegressor())
]
for name,model in models:
    run_model(name, model, train,y,test)

KNN F1 0.5517214506116731
LGBM F1 0.7340929540637331
XGB F1 0.6937436378138908


In [18]:
name = models[1][0]
model = models[1][1]
preds = run_model(name, model, train,y,test)

LGBM F1 0.7340929540637331


In [26]:
preds = [["Cheap","Affordable","Semi-Premium","Premium"][int(x)] for x in preds.round(0)]

In [27]:
sub = pd.DataFrame({"Property_Id":testId,"Price_Category":preds})

In [28]:
sub.to_csv("../output/sub_optuna1.csv",index=False)

In [30]:
pre_sub = pd.read_csv("../output/sub_with_sourcenum.csv")
pre_sub.Price_Category.value_counts()/len(pre_sub)

Semi-Premium    0.600000
Premium         0.359633
Affordable      0.040367
Name: Price_Category, dtype: float64

In [31]:
pre_sub2 = pd.read_csv("../output/sub3.csv")
pre_sub2.Price_Category.value_counts()/len(pre_sub)

Semi-Premium    0.598165
Premium         0.357798
Affordable      0.042202
Cheap           0.001835
Name: Price_Category, dtype: float64

In [32]:
sub["Price_Category"].value_counts()

Semi-Premium    324
Premium         199
Affordable       22
Name: Price_Category, dtype: int64

In [33]:
pd.concat([sub,pre_sub],axis=1)[sub.Price_Category != pre_sub.Price_Category]

,Property_Id,Price_Category,Property_Id,Price_Category
3,3000,Semi-Premium,3000,Premium
84,221,Semi-Premium,221,Affordable
124,739,Premium,739,Semi-Premium
141,973,Semi-Premium,973,Affordable
156,2103,Premium,2103,Semi-Premium
166,1258,Semi-Premium,1258,Premium
178,523,Semi-Premium,523,Premium
209,2293,Premium,2293,Semi-Premium
213,940,Semi-Premium,940,Affordable
278,2423,Premium,2423,Semi-Premium


In [47]:
pd.concat([sub,pre_sub2],axis=1)[sub.Price_Category != pre_sub2.Price_Category]

,Property_Id,Price_Category,Property_Id,Price_Category
52,711,Semi-Premium,711,Premium
84,221,Semi-Premium,221,Affordable
176,3612,Affordable,3612,Semi-Premium
195,451,Semi-Premium,451,Premium
269,2576,Semi-Premium,2576,Premium
282,1586,Premium,1586,Semi-Premium
299,125,Semi-Premium,125,Affordable
303,2692,Premium,2692,Semi-Premium
368,1408,Semi-Premium,1408,Premium
377,2329,Affordable,2329,Semi-Premium
